## Import necessary libraries

In [1]:
import pandas as pd
import requests
import numpy as np

## Load data into Pandas Dataframe

In [2]:
event_df = pd.read_csv('job_seeker_geo.csv')
event_df.head()

,id,created,eventType,eventValue,jobId,country,slug,geo
0,291A0388D8525FC7422BEB97C7D9481B,2022-08-22 15:30:46,Apply,10780,6301f4e68d484170669791ba,US,doordash,POINT(-122.04619996 47.62540000)
1,C5733B31E12896633B1A8FAEBE96DD47,2022-08-22 15:36:35,Apply,5179,63039b4b3bbdc2174316c1f6,US,doordash,POINT(-80.06789997 40.90490003)
2,53FDFAF42206E5607BC5424581F5EFDB,2022-08-22 15:38:17,Apply,10780,6301ce1e01883c93725f98d1,US,doordash,POINT(-122.22810002 47.56029999)
3,8C3AC70079C6EF595DCAB2EDFB904376,2022-08-22 15:56:08,Apply,4200,63039bf43bbdc2174316c654,US,doordash,POINT(-108.59680003 39.10710003)
4,84AF4C71DFCC270577140469C8901D04,2022-08-22 16:49:36,Apply,7909,63039c803bbdc2174316ca51,US,doordash,POINT(-86.79160002 36.15040004)


### Understand the dataset

In [3]:
event_df.keys()

Index(['id', 'created', 'eventType', 'eventValue', 'jobId', 'country', 'slug',
       'geo'],
      dtype='object')

In [4]:
event_df.count()

id            3575
created       3575
eventType     3575
eventValue    3575
jobId         3575
country       3575
slug          3575
geo           3575
dtype: int64

In [5]:
event_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3575 entries, 0 to 3574
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          3575 non-null   object
 1   created     3575 non-null   object
 2   eventType   3575 non-null   object
 3   eventValue  3575 non-null   int64 
 4   jobId       3575 non-null   object
 5   country     3575 non-null   object
 6   slug        3575 non-null   object
 7   geo         3575 non-null   object
dtypes: int64(1), object(7)
memory usage: 223.6+ KB


## Understand the API endpoint

In [18]:
DA_BASE_URL = 'https://us-state-api.herokuapp.com/?lat=3.7&lon=-119.67'

In [19]:
responseDA = requests.get(DA_BASE_URL)
print(responseDA.json())
print(responseDA.status_code)

{'state': None}
200


#### We need to make a call to the endpoint to get the state for all latitude-longtitude pairs in the `events_data.csv` file that is loaded into `event_df` dataframe

In [11]:
print(f"Totally {len(event_df)} rows, but only {len(event_df.geo.unique())} unique latitude-longitude pairs")

Totally 3575 rows, but only 1316 unique latitude-longitude pairs


####  Instead of making an API call for every row (3575 calls), let's make a call only for unique values in that column and use the response for every event row corresponding to that lat-long pair

In [13]:
#  list of unique states - lat long pairs

states = event_df.geo.unique()
print(states.size, states, )

1316 ['POINT(-122.04619996 47.62540000)' 'POINT(-80.06789997 40.90490003)'
 'POINT(-122.22810002 47.56029999)' ... 'POINT(-73.97809998 40.74429999)'
 'POINT(-81.66419996 41.38090002)' 'POINT(-97.31840000 32.72559997)']


#### Let's define a helper function to obtain the lat-long pair from the geo column containing lat-long in shapefile format

In [16]:
def getLatLong(geoVal):
    
    withoutPointPrefix = geoVal.split('(')
    latLong = (withoutPointPrefix[1]).split(' ')
    lng = latLong[1][:-1] 
    lat = latLong[0]
    return lng, lat

#### Getting the API response: 
Now, `states` list contains the unique lat-long pairs in `event_df`. In the following cell, we do three things for each element in this list:
- Create the request url by appending the lat-long pair (x',y') to the API base URL
- Invoke the API endpoint using `requests` library
- If we get a valid response, store the state and postal code from the response as a new column value for all rows with the same lat-long pair (x',y')

In [17]:

# i=0
for state in states:
    cordinates = getLatLong(state)
    payload = f"lat={cordinates[0]}&lon={cordinates[1]}"
    response = requests.get(DA_BASE_URL+payload).json()
    
    # print(response)
    if response['state'] != None:
        event_df.loc[(event_df['geo']==state), 'state'] = response['state']['slug'] 
        event_df.loc[(event_df['geo']==state), 'postal'] = response['state']['postal'] 
    else:
        event_df.loc[(event_df['geo']==state), 'state'] = np.nan
        event_df.loc[(event_df['geo']==state), 'postal'] = np.nan
#   for visibility on progress through the loop   
    # print(i) 
    # i+=1


In [19]:
event_df.head()

,id,created,eventType,eventValue,jobId,country,slug,geo,state,postal
0,291A0388D8525FC7422BEB97C7D9481B,2022-08-22 15:30:46,Apply,10780,6301f4e68d484170669791ba,US,doordash,POINT(-122.04619996 47.62540000),washington,WA
1,C5733B31E12896633B1A8FAEBE96DD47,2022-08-22 15:36:35,Apply,5179,63039b4b3bbdc2174316c1f6,US,doordash,POINT(-80.06789997 40.90490003),pennsylvania,PA
2,53FDFAF42206E5607BC5424581F5EFDB,2022-08-22 15:38:17,Apply,10780,6301ce1e01883c93725f98d1,US,doordash,POINT(-122.22810002 47.56029999),washington,WA
3,8C3AC70079C6EF595DCAB2EDFB904376,2022-08-22 15:56:08,Apply,4200,63039bf43bbdc2174316c654,US,doordash,POINT(-108.59680003 39.10710003),colorado,CO
4,84AF4C71DFCC270577140469C8901D04,2022-08-22 16:49:36,Apply,7909,63039c803bbdc2174316ca51,US,doordash,POINT(-86.79160002 36.15040004),tennessee,TN


#### Only 2 rows have null state and postal codes which is ~ 0.055% of the dataset, probably the states vermont and north-dakota as these US states are missing frmo observing manually. 

In [20]:
event_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3575 entries, 0 to 3574
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          3575 non-null   object
 1   created     3575 non-null   object
 2   eventType   3575 non-null   object
 3   eventValue  3575 non-null   int64 
 4   jobId       3575 non-null   object
 5   country     3575 non-null   object
 6   slug        3575 non-null   object
 7   geo         3575 non-null   object
 8   state       3573 non-null   object
 9   postal      3573 non-null   object
dtypes: int64(1), object(9)
memory usage: 279.4+ KB


### Storing the events data along with state information in a new csv file

In [22]:
event_df.to_csv('job_seeker_states.csv', index=False)